In [6]:
import pandas as pd
import cohere
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
df_str = pd.read_csv('https://raw.githubusercontent.com/winterForestStump/RNWF/main/data/rnwf_structure.csv', delimiter=';')
non_date_columns = ['Data']
date_columns = [col for col in df_str.columns if col not in non_date_columns]
date_columns = pd.to_datetime(date_columns, format='%d.%m.%Y')
last_date = date_columns.max() # Find the last available date
date_last_month = last_date - pd.DateOffset(months=1)
date_12_months_prior = last_date - pd.DateOffset(months=12) # Find the date 12 months prior
filtered_columns = [col for col in date_columns if col in [last_date, date_last_month, date_12_months_prior]]
filtered_columns_str = [col.strftime('%d.%m.%Y') for col in filtered_columns]
df_filtered = df_str[non_date_columns + filtered_columns_str]
df_filtered = df_filtered[df_filtered['Data'].isin(['Volume of the National Wealth Fund, RUB mln', 
                                                    'Volume of the National Wealth Fund, USD mln',
                                                    'Volume of liquid assets of the Fund, RUB mln',
                                                    'Volume of liquid assets of the Fund, USD mln',
                                                    'Share of liquid assets in the total Fund in USD equivalent'])]
df_filtered['Data'] = df_filtered['Data'].replace({
    'Volume of the National Wealth Fund, RUB mln': 'Total Volume (RUB mln)',
    'Volume of the National Wealth Fund, USD mln': 'Total Volume (USD mln)',
    'Volume of liquid assets of the Fund, RUB mln': 'Liquid Volume (RUB mln)',
    'Volume of liquid assets of the Fund, USD mln': 'Liquid Volume (USD mln)',
    'Share of liquid assets in the total Fund in USD equivalent': 'Liquid/Total in USD'
})
df_filtered.reset_index(drop=True, inplace=True)

In [4]:
df_filtered

,Data,01.12.2023,01.11.2024,01.12.2024
0,Total Volume (RUB mln),1.343297e+07,1.272620e+07,1.309672e+07
1,Total Volume (USD mln),1.511291e+05,1.311263e+05,1.215575e+05
2,Liquid Volume (RUB mln),6.746505e+06,5.447193e+06,5.792015e+06
3,Liquid Volume (USD mln),7.590530e+04,5.612600e+04,5.375870e+04
4,Liquid/Total in USD,5.022547e-01,4.280301e-01,4.422491e-01


In [16]:
dict(df_filtered)

{'Data': 0     Total Volume (RUB mln)
 1     Total Volume (USD mln)
 2    Liquid Volume (RUB mln)
 3    Liquid Volume (USD mln)
 4        Liquid/Total in USD
 Name: Data, dtype: object,
 '01.12.2023': 0    1.343297e+07
 1    1.511291e+05
 2    6.746505e+06
 3    7.590530e+04
 4    5.022547e-01
 Name: 01.12.2023, dtype: float64,
 '01.11.2024': 0    1.272620e+07
 1    1.311263e+05
 2    5.447193e+06
 3    5.612600e+04
 4    4.280301e-01
 Name: 01.11.2024, dtype: float64,
 '01.12.2024': 0    1.309672e+07
 1    1.215575e+05
 2    5.792015e+06
 3    5.375870e+04
 4    4.422491e-01
 Name: 01.12.2024, dtype: float64}

In [7]:
COHERE_API = os.getenv('COHERE_API_KEY')
cohere_client = cohere.ClientV2(api_key=COHERE_API)

In [22]:
system_message = '''
You are a financial analyst. You will be provided with a table containing financial data. Your task is to write a short, concise, and professional report analyzing only the table data. The report should address the following questions:
1. What were the dynamics of the Total Volume in RUB over the last year and the last month?
2. What were the dynamics of the Total Volume in USD over the last year and the last month?
3. What were the dynamics of the Liquid Volume in RUB over the last year and the last month?
4. What were the dynamics of the Liquid Volume in USD over the last year and the last month?
5. How did the share change over the last year and the last month?
You have to convert the numbers from scientific notation to regular.
'''

In [23]:
response = cohere_client.chat(model="command-r-plus-08-2024", 
                              messages=[{"role": "system", "content": system_message},
                                        {"role": "user", "content": str(df_filtered),},
                                       ],
                              temperature = 0.5,
                             )

In [24]:
report = response.message.content[0].text.split('\n')

In [25]:
report

['## Financial Data Analysis Report',
 '',
 '### Total Volume in RUB:',
 'The Total Volume in Russian Rubles (RUB) demonstrated a slight increase of approximately 3.7% from 1.272620e+07 RUB mln in November 2023 to 1.309672e+07 RUB mln in December 2023. However, when compared to the data from a year earlier (December 2023), the volume decreased by around 2.5%.',
 '',
 '### Total Volume in USD:',
 'In terms of US Dollars (USD), the Total Volume experienced a decline over both periods. From November to December 2023, it decreased by roughly 7.3%, and the year-over-year comparison shows a more significant drop of 19.6%.',
 '',
 '### Liquid Volume in RUB:',
 'The Liquid Volume in RUB saw an increase of 6.3% from November to December 2023, rising from 5.447193e+06 RUB mln to 5.792015e+06 RUB mln. However, compared to December 2023, it decreased by 12.4%.',
 '',
 '### Liquid Volume in USD:',
 'In USD terms, the Liquid Volume decreased by 4.2% from 5.612600e+04 USD mln to 5.375870e+04 USD mln 